In [1]:
from stock_article import StockArticle
from utility import HTMLParser
from utility import FileSaver

stock_article = StockArticle()

news = stock_article.fetch_news("Google", news_count=10)

html_parser = HTMLParser()
file_saver = FileSaver()
for item in news:
    html_parser.parse_html(item["link"])
    file_name = html_parser.get_title() + ".txt"
    parsed_content = html_parser.get_paragraphs()
    file_saver.save_to_file(parsed_content, file_name)


Parsing HTML from: https://finance.yahoo.com/m/b538a944-b109-341f-b1fc-c2fdd585881b/google-employees-are-testing.html
Saving to file: output/google_employees_are_testing_new_ai_mode_search_feature.txt
Parsing HTML from: https://finance.yahoo.com/m/3412ea55-83f4-31bf-bad8-420eedf033d7/google-messages-might-let-you.html
Saving to file: output/google_messages_might_let_you_unsend_awkward_messages_in_rcs_chats.txt
Parsing HTML from: https://finance.yahoo.com/m/13ae0e9c-7fd1-31d0-858d-4b3358c4350d/byte-sized-ai%3A-ai-in-google%E2%80%99s.html
Saving to file: output/byte-sized_ai:_ai_in_google’s_earnings,_more_supply_chain_tech_upgrades.txt
Parsing HTML from: https://finance.yahoo.com/m/33a0582f-4501-3678-9994-0340c5200255/you-can-officially-download.html
Saving to file: output/you_can_officially_download_the_tiktok_app_again_on_android_phones.txt
Parsing HTML from: https://finance.yahoo.com/m/8c427a33-89c6-4d0e-9dac-66764d87cd5f/google%27s-pixel-buds-pro-2-are.html
Saving to file: output/goo